In [31]:
!pip install transformers

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [21]:
import sagemaker
import numpy as np
import pandas as pd
import os
import torch
from sagemaker.pytorch.model import PyTorchModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [3]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

## Create model from files on S3

In [32]:
from transformers import RobertaForTokenClassification

In [34]:
model = RobertaForTokenClassification.from_pretrained('s3://sagemaker-eu-west-1-087816224558/sagemaker-pytorch-2020-05-31-12-23-54-436/output/model.tar.gz',num_labels=20)

OSError: Can't load 's3://sagemaker-eu-west-1-087816224558/sagemaker-pytorch-2020-05-31-12-23-54-436/output/model.tar.gz'. Make sure that:

- 's3://sagemaker-eu-west-1-087816224558/sagemaker-pytorch-2020-05-31-12-23-54-436/output/model.tar.gz' is a correct model identifier listed on 'https://huggingface.co/models'

- or 's3://sagemaker-eu-west-1-087816224558/sagemaker-pytorch-2020-05-31-12-23-54-436/output/model.tar.gz' is the correct path to a directory containing a 'config.json' file



## Deploy model

In [17]:
input_data = pd.read_csv(os.path.join('../data/processed_roberta/', 'test_roberta.csv'), header=None, names=None)

In [22]:
# ---- UNPACK DATA ----
val_tags = input_data[input_data.columns[0:45]].values
val_inputs = input_data[input_data.columns[45:90]].values
val_masks = input_data[input_data.columns[90:135]].values
val_segs = input_data[input_data.columns[135:]].values

val_inputs = torch.tensor(val_inputs)
val_tags = torch.tensor(val_tags)
val_masks = torch.tensor(val_masks)
val_segs = torch.tensor(val_segs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=32)

In [27]:
pytorch_model = PyTorchModel(
    model_data='s3://sagemaker-eu-west-1-087816224558/sagemaker-pytorch-2020-05-31-12-23-54-436/output/model.tar.gz',
    entry_point='predict.py',
    source_dir="serve",
    role=role,
    py_version='py3',
    framework_version='1.1.0')

In [28]:
predictor = pytorch_model.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1)

---------------!

In [29]:
predictions = predictor.predict(valid_dataloader)

ConnectionClosedError: Connection was closed before we received a valid response from endpoint URL: "https://runtime.sagemaker.eu-west-1.amazonaws.com/endpoints/sagemaker-pytorch-2020-05-31-14-45-09-224/invocations".

In [ ]:
print(predictions)